In [7]:
import os
import fitz  # PyMuPDF
import json
import re

pdf_path = "اخلاق کار تشکیلاتی.pdf"
book_title = "اخلاق کار تشکیلاتی"
output_dir = "ChunkedBook"
output_path = f"{output_dir}/اخلاق_کار_تشکیلاتی_chunks.jsonl"

# ساخت پوشه اگر وجود ندارد
os.makedirs(output_dir, exist_ok=True)

doc = fitz.open(f"Books/{pdf_path}")
chunks = []

current_chapter = "مقدمه"

def clean_text(text):
    # حذف فوترهای تکراری
    junk_phrases = [
        "هسسوم ناناوج ناتسآ سدق یوضر",  # وارونه
        "رضوی قدس آستان جوانان موسسه",  # اصل
        "اخالق کارِ تشکیالتی",
    ]
    for junk in junk_phrases:
        text = text.replace(junk, "")
    # حذف کشیدگی‌های غیرطبیعی: سه یا بیشتر از یک حرف تکراری
    text = re.sub(r"([اآبپتثجچحخدذرزژسشصضطظعغفقکگلمنوهی])\u200c{2,}", r"\1", text)
    # حذف فاصله‌های زیاد بین حروف
    text = re.sub(r"(\w)\s{2,}(\w)", r"\1 \2", text)
    # حذف کشیدگی‌های تزئینی (مثلاً ســؤال → سؤال)
    text = re.sub(r"([اآبپتثجچحخدذرزژسشصضطظعغفقکگلمنوهی])\s*ـ+\s*", r"\1", text)
    # تبدیل چندین \n به یک \n\n برای حفظ پاراگراف‌ها
    text = re.sub(r"\n{3,}", "\n\n", text)
    # تبدیل \n وسط جمله به فاصله ساده
    text = re.sub(r"(?<!\n)\n(?!\n)", " ", text)

    return text.strip()

In [8]:
for page_num, page in enumerate(doc, start=1):
    text = page.get_text("text").strip()
    text = clean_text(text)
    # تشخیص عنوان فصل (ساده: اگر خط اول بزرگ و جدا باشد)
    lines = text.split("\n")
    if len(lines) > 0 and len(lines[0]) < 50 and lines[0].isalpha():
        for line in lines[:5]:
            if "فصل" in line or "بخش" in line:
                current_chapter = line.strip()
                break

    chunk = {
        "book": book_title,
        "chapter": current_chapter,
        "page": page_num,
        "text": text,
    }
    chunks.append(chunk)

# ذخیره در قالب JSONL
with open(output_path, "w", encoding="utf-8") as f:
    for chunk in chunks:
        f.write(json.dumps(chunk, ensure_ascii=False) + "\n")

print(f"✅ استخراج کامل شد. تعداد صفحات: {len(chunks)}. خروجی: {output_path}")

✅ استخراج کامل شد. تعداد صفحات: 79. خروجی: ChunkedBook/اخلاق_کار_تشکیلاتی_chunks.jsonl
